<h1>Building Dataset From Riot API</h1>

In [11]:
import json
import requests
import warnings
import time
import pickle

In [2]:
file = open("riot_api.json", 'r')

# Open the credentials file with json.load
credentials = json.load(file)
file.close()

In [3]:
request_url = f"https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/IRON/IV?page=1&api_key={credentials["key"]}"
resp = requests.get(request_url)
iron4_summoner_ids = []
if (resp.status_code == 200):
    print(resp.json()[0])
    for summoner in resp.json():
        iron4_summoner_ids.append(summoner["summonerId"])
else:
    warnings.warn(f"Error: Response Code {resp.status_code}")

print(iron4_summoner_ids)

{'leagueId': '18463ac3-0074-4d69-a73c-53babb6dab30', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'IRON', 'rank': 'IV', 'summonerId': 'sSkS9fw41Tu9Ty4w8sWisy_BLnuMyiByyx6FD8pHsfAbu7-M', 'summonerName': 'Useless Ginger', 'leaguePoints': 56, 'wins': 15, 'losses': 36, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
['sSkS9fw41Tu9Ty4w8sWisy_BLnuMyiByyx6FD8pHsfAbu7-M', '_NQ3F3_mj5c9vptUvqwsaS5J84s4_n8Zg5AsS9P-O0iw86Gm', 'Ultef8EUDiEUoGcfdiVbt1h8k4bSP-ISRY40Vr78hdy6a_nt', '6u7CD0PvFECs29aqQrHyNMkJP8vjwlF94RXdYTcOSVWVZhhf', 'UOwkW8Vn4WhaPbZ3kLNn59VAQ3uQe207yoHiJvoJlvsjNSug', 'AC6_dDolZvvnlKyZJ-nYMdYevpERNI-yAKXo9tsUV0RBZMYu', 'Wln4fAyzNKZazzjO5BeAPAKCgzIjYo2sd0qGOO_bYVjAz32V', 'W_vbM3HnQgzlkxiKXxiPjUyE0rmW-a_f_whzxANAwG2QKhQv', 'emzZZNkwt05djO2wqgl01Ptd9u4O_JgI-fYc06Hcg2cjl89l', 'C8FdGHYGvC2xwcLHAVOj86v6dNGsGI3JtcyroBDkvm27oQpz', '-ZgGzLb2bJKAMhU29VeJ2pNQnQihY9oMRm3JzQTe6MkT9al4', '8Sjub9LCmyp0gte4V7Pd1qOzWyUhRDuEjPVfWsNnY5V8pvWj', 'ZtWuTdbyu6Rz9PSWuhOSacRlLPZ4ywEgIoe

In [10]:
# API LIMITS
# 20 requests every second
# 100 requests every 120 seconds
short_request_count = 1
long_request_count = 3
short_request_max = 20
long_request_max = 100
short_request_cooldown = 1
long_request_cooldown = 120
short_request_start_time = time.perf_counter()
long_request_start_time = time.perf_counter()
def check_requests(short_request_count, long_request_count, short_request_max, long_request_max, short_request_cooldown, long_request_cooldown, short_request_start_time, long_request_start_time):
    short_request_count = short_request_count
    long_request_count = long_request_count
    if time.perf_counter() - long_request_start_time < long_request_cooldown and long_request_count >= long_request_max:
        print(f"Sleeping for: {long_request_cooldown - (time.perf_counter() - long_request_start_time):.2f} seconds")
        time.sleep(long_request_cooldown - (time.perf_counter() - long_request_start_time))
        long_request_start_time = time.perf_counter()
        short_request_start_time = time.perf_counter()
        long_request_count = 1
        short_request_count = 1
    elif time.perf_counter() - long_request_start_time > long_request_cooldown:
        print("Restarting long request")
        long_request_start_time = time.perf_counter()
        long_request_count = 1
    elif time.perf_counter() - short_request_start_time < short_request_cooldown and short_request_count >= short_request_max:
        print(f"Sleeping for: {short_request_cooldown - (time.perf_counter() - short_request_start_time):.2f} seconds")
        time.sleep(short_request_cooldown - (time.perf_counter() - short_request_start_time))
        short_request_start_time = time.perf_counter()
        short_request_count = 1
    elif time.perf_counter() - short_request_start_time > short_request_cooldown:
        print("Restarting short request")
        short_request_start_time = time.perf_counter()
        short_request_count = 1

total_match_ids = []

for summoner_id in iron4_summoner_ids:
    get_summoner_request_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={credentials["key"]}"
    resp = requests.get(get_summoner_request_url)
    puuid = None
    if (resp.status_code == 200):
        puuid = resp.json()["puuid"]
    else:
        warnings.warn(f"Error: Response Code {resp.status_code}")
        if resp.status_code == 429:
            print("Rate limit exceeded. Sleeping for 120 seconds")
            time.sleep(120)
            request_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={credentials["key"]}"
            resp = requests.get(request_url)
            if (resp.status_code == 200):
                puuid = resp.json()["puuid"]
            short_request_count = 1
            long_request_count = 1
            short_request_time = time.perf_counter()
            long_request_time = short_request_time
    
    short_request_count += 1
    long_request_count += 1
    print(f"Short request count: {short_request_count}")
    print(f"Long request count: {long_request_count}")
    check_requests(short_request_count, long_request_count, short_request_max, long_request_max, short_request_cooldown, long_request_cooldown, short_request_start_time, long_request_start_time)
    match_id = None
    if puuid != None:
        get_matches_request_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10&api_key={credentials["key"]}"
        resp = requests.get(get_matches_request_url)
        if (resp.status_code == 200):
            for match in resp.json():
                total_match_ids.append(match)
        else:
            warnings.warn(f"Error: Response Code {resp.status_code}")
            if resp.status_code == 429:
                print("Rate limit exceeded. Sleeping for 120 seconds")
                time.sleep(120)
                get_matches_request_url = get_matches_request_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10&api_key={credentials["key"]}"
                resp = requests.get(get_matches_request_url)
                if (resp.status_code == 200):
                    for match in resp.json():
                        total_match_ids.append(match)
                short_request_count = 1
                long_request_count = 1
                short_request_time = time.perf_counter()
                long_request_time = short_request_time
        
        short_request_count += 1
        long_request_count += 1
        print(f"Short request count: {short_request_count}")
        print(f"Long request count: {long_request_count}")
        check_requests(short_request_count, long_request_count, short_request_max, long_request_max, short_request_cooldown, long_request_cooldown, short_request_start_time, long_request_start_time)

print("********************FINISHED**********************")
print(total_match_ids)

Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 4
Long request count: 6
Restarting short request
Short request count: 5
Long request count: 7
Restarting short request
Short request count: 6
Long request count: 8
Restarting short request
Short request count: 7
Long request count: 9
Restarting short request
Short request count: 8
Long request count: 10
Restarting short request
Short request count: 9
Long request count: 11
Restarting short request
Short request count: 10
Long request count: 12
Restarting short request
Short request count: 11
Long request count: 13
Restarting short request
Short request count: 12
Long request count: 14
Restarting short request
Short request count: 13
Long request count: 15
Restarting short request
Short request count: 14
Long request count: 16
Restarting short request
Short request count: 15
Long request count: 17
Restarting short request
Short request count: 16
Long re

C:\Users\dylan\AppData\Local\Temp\ipykernel_20856\2249859885.py:45: UserWarning: Error: Response Code 429
  warnings.warn(f"Error: Response Code {resp.status_code}")


Rate limit exceeded. Sleeping for 120 seconds
Short request count: 2
Long request count: 2
Restarting long request
Short request count: 3
Long request count: 3
Restarting long request
Short request count: 4
Long request count: 4
Restarting long request
Short request count: 5
Long request count: 5
Restarting long request
Short request count: 6
Long request count: 6
Restarting long request
Short request count: 7
Long request count: 7
Restarting long request
Short request count: 8
Long request count: 8
Restarting long request
Short request count: 9
Long request count: 9
Restarting long request
Short request count: 10
Long request count: 10
Restarting long request
Short request count: 11
Long request count: 11
Restarting long request
Short request count: 12
Long request count: 12
Restarting long request
Short request count: 13
Long request count: 13
Restarting long request
Short request count: 14
Long request count: 14
Restarting long request
Short request count: 15
Long request count: 15


In [14]:
file_path = 'data.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(total_match_ids, file)

In [15]:
loaded_data = None

with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)